In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  445  0  0   1060   1061  1059  0.01  0.36  0.45  0.79  0.78
1  475  1  0   1014   1018  1017  0.01  0.38  0.45  0.81  0.81
2  481  2  0    983    989   987  0.02  0.39  0.45  0.80  0.80
3  484  3  0    959    967   966  0.02  0.41  0.45  0.78  0.77
4  487  4  0    938    948   947  0.02  0.43  0.45  0.75  0.74
5  505  5  0    923    934   933  0.03  0.44  0.45  0.78  0.78
6  503  6  0    914    923   923  0.03  0.44  0.45  0.77  0.77
7  503  7  0    907    916   916  0.03  0.44  0.45  0.77  0.77
8  510  8  0    903    912   912  0.03  0.43  0.45  0.77  0.77

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta20-gam0.01-epoch50/log' mode='wb' closefd=True>


 0 11258 487953.0 write use one get article know like make think say
   0-1 7091 329850.0 say people go write one think get article make year
     0-1-1 6974 6479.0 san helmet double american paint probably francisco information meeting member
     0-1-2 117 94.0 air wave pressure transfer europe protect manage original scale thread
   0-2 3416 134956.0 file use program key window available information image system space
     0-2-1 3395 1779.0 pen reserve appreciate installation shipping doug function chapter hus school
     0-2-2 21 4.0 adapter uk xt bhj final currently various legal washington throw
   0-3 215 16417.0 pt game team vs new la play period hockey win
     0-3-1 214 120.0 catholic frank joe bill jim car mike annual kevin steve
     0-3-2 1 0.0 bhj series ram dave knowledge prevent james throw washington legal
   0-4 20 2315.0 db mov bh cs rg byte oo al bit eus
     0-4-1 20 47.0 mb mr pl ma oo hp id mi nice us
   0-5 507 8957.0 ground wire water circuit wiring outlet neut